# Graz Population

In [5]:
!pip install rasterio -q

In [12]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
from shapely.geometry import Point
import numpy as np

1. Population clip to Graz

In [10]:
graz = gpd.read_file("graz.shp")

with rasterio.open('aut_ppp_2020.tif') as src:
    # Schneiden Sie das Raster auf die Maske zu
    out_image, out_transform = mask(src, graz.geometry, crop=True)
    out_meta = src.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

with rasterio.open('aut_ppp_2020_graz.tif', 'w', **out_meta) as dest:
    dest.write(out_image)

2. Raster to Vector - Point

In [20]:
with rasterio.open('aut_ppp_2020_graz.tif') as src:
    image = src.read(1)
    transform = src.transform 
    
    nodata_value = src.nodata

    points = []
    values = []
    
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            value = image[row, col]
            if value != nodata_value:
                x, y = transform * (col, row)
                points.append(Point(x, y))
                values.append(value)
    
    graz_pop = gpd.GeoDataFrame({'value': values}, geometry=points, crs=src.crs)

graz_pop.to_parquet('graz_pop.parquet')